In [1]:
#Bibliotecas
source('utils.r',local=TRUE)

Loading required package: lattice
Loading required package: ggplot2
Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following object is masked from ‘package:glmnet’:

    auc

The following objects are masked from ‘package:stats’:

    cov, smooth, var



In [2]:
#Leitura do dataset já tratado
data_frame = read_dataset(split=FALSE)

In [3]:
#Modelos alternativos
alternative1 = glm(formula = default ~ t_emprego + t_endereco + div + renda + renda:t_emprego, family = binomial(link = logit), data = data_frame)
alternative2 = glm(formula = default ~ t_emprego + t_endereco + div + renda + renda:div, family = binomial(link = logit), data = data_frame)

#Coeficientes dos modelos
coef(alternative1)
coef(alternative2)

#Note que as estimativas dos coeficientes de cada modelo são diferentes
#Isso é causado pela interação.
#É interessante notar, por exemplo, a diferença de renda nos modelos. 
#Enquanto que em alternative1 esse valor é razoavelmente baixo, o modelo
#alternative2 dá um peso muito grande a ele.
#Novamente salientamos que os tomadores de decisão precisão avaliar qual
#variável deve possuir um peso maior, possibilitando escolher o modelo adequado.

(Intercept)       t_emprego      t_endereco             div           renda 
      3.6481750       2.1164222       0.4870481      -1.6296178       0.6669342 
t_emprego:renda 
     -0.4005257

(Intercept)   t_emprego  t_endereco         div       renda   div:renda 
  1.4272015   1.3916426   0.4887177  -0.8227885   2.3356122  -0.5533865

In [4]:
#Efeito dos parâmetros com IC
odds1 = exp(summary(alternative1)$coefficients[,1] + 
(matrix(rep(qnorm(c(0.025,0.5,0.975)),6),nrow=6,byrow=TRUE) * summary(alternative1)$coefficients[,2]))

odds2 = exp(summary(alternative2)$coefficients[,1] + 
(matrix(rep(qnorm(c(0.025,0.5,0.975)),6),nrow=6,byrow=TRUE) * summary(alternative2)$coefficients[,2]))

#Mudando nome das colunas
colnames(odds1) = c('ICfloor','odss','ICceil')
colnames(odds2) = c('ICfloor','odss','ICceil')

#Criando dataframe
odds1 = data.frame(odds1,row.names = names(alternative1$coefficients))[-1,]
odds2 = data.frame(odds2,row.names = names(alternative2$coefficients))[-1,]

#Arredondando valores
odds1 = round(odds1,3)
odds2 = round(odds2,3)

#Apresentando resultado
odds1
odds2

#Note pelos valores calculados que dependendo do modelo e da iteração as variáveis possuem efeitos
#completamente diferentes quanto a magnitude. Desse modo, cabe ao tomadores de decisão escolher 
#qual variável possuirá um peso maior no seu problema. 
#Perceba também que todos os ODDS estão dentro do IC calculado.
#Para maiores detalhes sobre a análise verifique o relatório em anexo neste projeto.

,ICfloor,odss,ICceil
t_emprego,4.177,8.301,16.497
t_endereco,1.222,1.628,2.168
div,0.130,0.196,0.295
renda,1.200,1.948,3.162
t_emprego:renda,0.528,0.670,0.851


,ICfloor,odss,ICceil
t_emprego,2.624,4.021,6.164
t_endereco,1.213,1.630,2.192
div,0.258,0.439,0.746
renda,3.262,10.336,32.751
div:renda,0.433,0.575,0.763
